# The aim of this notebook is to answer following questions on the given dataset.

Using data in file 'data/world_bank_projects.json'
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

## import libraries

In [72]:
import pandas as pd

import json
from pandas.io.json import json_normalize

## import data

In [73]:
# load as dataframe
df_world = pd.read_json("data/world_bank_projects.json")

# load as json string
js_world = json.load((open('data/world_bank_projects.json')))

## get an overview of the data

In [80]:
df_world.head()

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{'Percent': 100, 'Name': 'Education for all'}","[{'code': '65', 'name': 'Education for all'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{'$oid': '52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{'Percent': 30, 'Name': 'Other economic manage...","[{'code': '24', 'name': 'Other economic manage...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en
2,{'$oid': '52b213b38594d8a2be17c782'},2014,November,2013-11-01T00:00:00Z,MINISTRY OF FINANCE AND ECONOMIC DEVEL,NaN,Tuvalu!$!TV,TV,Tuvalu,Tuvalu,...,TI,IBRD,Active,Y,"{'Percent': 46, 'Name': 'Regional integration'}","[{'code': '47', 'name': 'Regional integration'...","52,81,25,47",6060000,6060000,http://www.worldbank.org/projects/P145310?lang=en
3,{'$oid': '52b213b38594d8a2be17c783'},2014,October,2013-10-31T00:00:00Z,MIN. OF PLANNING AND INT'L COOPERATION,NaN,Republic of Yemen!$!RY,RY,Republic of Yemen,"Yemen, Republic of",...,JB,IBRD,Active,N,"{'Percent': 50, 'Name': 'Participation and civ...","[{'code': '57', 'name': 'Participation and civ...","59,57",0,1500000,http://www.worldbank.org/projects/P144665?lang=en
4,{'$oid': '52b213b38594d8a2be17c784'},2014,October,2013-10-31T00:00:00Z,MINISTRY OF FINANCE,2019-04-30T00:00:00Z,Kingdom of Lesotho!$!LS,LS,Kingdom of Lesotho,Lesotho,...,"FH,YW,YZ",IBRD,Active,N,"{'Percent': 30, 'Name': 'Export development an...","[{'code': '45', 'name': 'Export development an...","41,45",13100000,13100000,http://www.worldbank.org/projects/P144933/seco...


In [81]:
df_world["mjtheme_namecode"].head()

0    [{'code': '8', 'name': 'Human development'}, {...
1    [{'code': '1', 'name': 'Economic management'},...
2    [{'code': '5', 'name': 'Trade and integration'...
3    [{'code': '7', 'name': 'Social dev/gender/incl...
4    [{'code': '5', 'name': 'Trade and integration'...
Name: mjtheme_namecode, dtype: object

In [82]:
df_world["mjtheme_namecode"].loc[2]

[{'code': '5', 'name': 'Trade and integration'},
 {'code': '2', 'name': 'Public sector governance'},
 {'code': '11', 'name': 'Environment and natural resources management'},
 {'code': '6', 'name': 'Social protection and risk management'}]

## 1. Find the 10 countries with most projects

In [83]:
country_names = df_world.countryshortname.value_counts()
country_names[0:10]

Indonesia             19
China                 19
Vietnam               17
India                 16
Yemen, Republic of    13
Morocco               12
Bangladesh            12
Nepal                 12
Mozambique            11
Africa                11
Name: countryshortname, dtype: int64

## 2. Find the top 10 major project themes (using column 'mjtheme_namecode')

In [84]:
# Copy dataset and normalize
js_world_c = js_world.copy()
js_world_n = json_normalize(js_world_c, "mjtheme_namecode")

# create dictionary for code - name pairs
dict_code_name = js_world_n[js_world_n["name"] != ""]
dict_code_name = dict_code_name.dropna()
dict_code_name = dict_code_name.drop_duplicates()
dict_code_name = pd.Series(dict_code_name.name.values, index=dict_code_name.code).to_dict()

# replace missing values with code - name paris
js_world_n.loc[js_world_n.name == "", "name"] = js_world_n.loc[js_world_n.name == "", "code"]
js_world_n = js_world_n.replace({ "name" : dict_code_name})
js_world_n

js_world_n.name.value_counts()[0:10]

Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
Name: name, dtype: int64

## 3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [85]:
# Copy dataset and normalize
js_world_c = js_world.copy()
js_world_n = json_normalize(js_world_c, "mjtheme_namecode", [["_id", "$oid"]])
js_world_n.columns = ["code", "name", "project_id"]

# Fill missing name values
js_world_n.loc[js_world_n.name == "", "name"] = js_world_n.loc[js_world_n.name == "", "code"]
js_world_n = js_world_n.replace({ "name" : dict_code_name})
js_world_n

js_world_n

,code,name,project_id
0,8,Human development,52b213b38594d8a2be17c780
1,11,Environment and natural resources management,52b213b38594d8a2be17c780
2,1,Economic management,52b213b38594d8a2be17c781
3,6,Social protection and risk management,52b213b38594d8a2be17c781
4,5,Trade and integration,52b213b38594d8a2be17c782
5,2,Public sector governance,52b213b38594d8a2be17c782
6,11,Environment and natural resources management,52b213b38594d8a2be17c782
7,6,Social protection and risk management,52b213b38594d8a2be17c782
8,7,Social dev/gender/inclusion,52b213b38594d8a2be17c783
9,7,Social dev/gender/inclusion,52b213b38594d8a2be17c783
